In [1]:
from utils.util import get_project_root
from agents.renderer.test_prompt import prompt,org_text

In [2]:
from utils.llm import LLMProfile
from langchain_core.output_parsers import JsonOutputParser
import json 
final_prompt = prompt.replace("{RAW_MARKDOWN_REPORT}", org_text)
slice_plan_llm = LLMProfile.cloud_llm().invoke(final_prompt)
slice_plan_json = JsonOutputParser().parse(slice_plan_llm)


/Users/seobi/PythonProjects/RAG_Commander/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: This Anthropic LLM is deprecated. Please use `from langchain_anthropic import ChatAnthropic` instead
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [3]:
print(slice_plan_llm)

```json
{
  "meta": {
    "title": "서울 강남구 역삼동 84㎡ 120세대 분양성 평가",
    "theme": "dark_gold",
    "renderer": "google_slides_simple",
    "compat": ["title", "bluf", "text", "table"]
  },
  "slides": [
    {
      "type": "text",
      "title": "1. 사업환경 분석",
      "lead": "교통·교육·편의 3요건 검증 필요, 정책 호재는 중장기 우호적이나 단기 공급 확대로 가격 민감도 상승",
      "groups": [
        {
          "label": "교육 인구(강남구, 2025.09)",
          "items": [
            "10~19세 64,171명(2025.04~09 +597명, +0.9%)",
            "0~9세 32,142명(동기 -628명, -1.9%)",
            "중·고 학령층 유지로 학군 선호형 수요 견조"
          ]
        },
        {
          "label": "교통·생활편의",
          "items": [
            "최근접 지하철역(2·9·수인분당선) 실보행 미확인",
            "도보 7~10분 역세권 확보 시 상단가 정당화",
            "12분 이상 시 설득력 약화"
          ]
        },
        {
          "label": "인근 입주 예정(2025~2026)",
          "items": [
            "청담 르엘 1,261세대(2025.11)",
            "디에이치 대치 에델루이 282세대(2025.08)",
            "래미안 레벤투스 308세대(2026.10)",
            "단기 대체재 확대로 초

In [4]:
from agents.renderer.renderer_logic import render_sliceplan
render_sliceplan(slice_plan_json)

{'presentationId': '1-gKowVc8dKoQlD54QfQ7RzDyOjePFJOMucrwRFJihlw',
 'url': 'https://docs.google.com/presentation/d/1-gKowVc8dKoQlD54QfQ7RzDyOjePFJOMucrwRFJihlw/edit'}